In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys, os , argparse

# Constant Parameters for CFP modeling
The following parameters are constants, representing the maximun, minium and most probable values sampled from our probabilistic distribution. If you re-run the **probabilistic sequence generation** in `cfp_calculation.ipynb`, you may obtain slightly different values.


# Read and clean the base dataset

In [4]:
processors = pd.read_excel('DataSetWithRelaventNodesArea.xlsx')
processors = processors.dropna(subset=['DieSizeValue', 'Die Size (mm^2)'], how='all')
processors = processors[(processors['DieSizeValue'] != 0) | (processors['Die Size (mm^2)'] != 0)]
processors = processors[processors['DieSizeValue'] != 'unknown']
processors = processors[processors['Die Size (mm^2)'] != 'unknown']
# Chiplet processors have different notations in die size value and die size
chiplet = processors[processors['DieSizeValue'] != processors['Die Size (mm^2)']]


processors = processors.drop(processors[processors['DieSizeValue'] != processors['Die Size (mm^2)']].index)
processors = processors[processors['TDP (W)'].notna()]
processors = processors[processors['TDP (W)'] != 'unknown']
processors = processors[processors['TDP (W)'] != '0']

In [5]:
# most probable values of input parameters
nodes = ['7', '10', '14', '22', '28']
ci = 540
gpa = 150
epa = {'7': 0.235, '10': 0.195, '14': 0.125, '22': 0.140, '28': 0.105}
defect_density = {'7': 0.224, '10': 0.147, '14': 0.127, '22': 0.091, '28': 0.081}


In [6]:
# minimum values of input parameters
ci_min = 480.849
gpa_min = 50
defect_density_min = {'7': 0.185, '10': 0.095, '14': 0.075, '22': 0.065, '28': 0.055}
epa_min = {'7': 0.185, '10': 0.095, '14': 0.075, '22': 0.065, '28': 0.055}


In [7]:
# maximum values of input parameters
ci_max = 544.644
gpa_max = 252.533
defect_density_max = {'7': 0.497, '10': 0.407, '14': 0.387, '22': 0.377, '28': 0.367}
epa_max = {'7': 0.485, '10': 0.395, '14': 0.375, '22': 0.365, '28': 0.355}

In [8]:
sys.path.append(os.path.join(os.getcwd(), "../eco_chip_enhanced"))

from eco_chip_func import eco_chip
for idx, proc in processors.iterrows():

    # node = int(proc['Process Size (nm)'])
    node = int(proc['Process Size (nm)'])
    power = int(proc['TDP (W)'])
    area = int(proc['DieSizeValue'])

    defect = defect_density_min[str(node)]
    epa_val = epa_min[str(node)]
    gpa_val =gpa_min
    ci_val = ci_min

    args = argparse.Namespace(design_dir='testcases/CFP_survey/', 
                                chip_area=area, 
                                node=node, 
                                chip_power=power, 
                                defect_density=defect,
                                num_lifetime=None, 
                                gpa = gpa_val, 
                                epa = epa_val,
                                tech_scaling_path='../eco_chip_enhanced/',
                                carbon_intensity=ci_val)    
    c_des, c_mfg, c_ope, c_tot = eco_chip(args)
    min_emb = float(c_des) + float(c_mfg)
    min_ope = c_ope
    min_tot = min_emb + float(c_ope)
    processors.at[idx, 'Emb Crabon Min'] = min_emb
    processors.at[idx, 'Ope Carbon Min'] = min_ope
    processors.at[idx, 'Tot Carbon Min'] = min_tot
    processors.at[idx, 'ECFPA Min'] = min_emb / area


    defect = defect_density_max[str(node)]
    epa_val = epa_max[str(node)]
    gpa_val =gpa_max
    ci_val = ci_max

    args = argparse.Namespace(design_dir='testcases/CFP_survey/', 
                                chip_area=area, 
                                node=node, 
                                chip_power=power, 
                                defect_density=defect,
                                num_lifetime=None, 
                                gpa = gpa_val, 
                                epa = epa_val,
                                tech_scaling_path='../eco_chip_enhanced/',
                                carbon_intensity=ci_val)    
    c_des, c_mfg, c_ope, c_tot = eco_chip(args)
    max_emb = float(c_des) + float(c_mfg)
    max_ope = c_ope
    max_tot = max_emb + float(c_ope)

    processors.at[idx, 'Emb Crabon Max'] = max_emb
    processors.at[idx, 'Ope Carbon Max'] = max_ope
    processors.at[idx, 'Tot Carbon Max'] = max_tot
    processors.at[idx, 'ECFPA Max'] = max_emb / area

    defect = defect_density[str(node)]
    epa_val = epa[str(node)]
    gpa_val =gpa
    ci_val = ci

    args = argparse.Namespace(design_dir='testcases/CFP_survey/', 
                                    chip_area=area, 
                                    node=node, 
                                    chip_power=power, 
                                    defect_density=defect,
                                    num_lifetime=None, 
                                    gpa = gpa_val, 
                                    epa = epa_val,
                                    tech_scaling_path='../eco_chip_enhanced/',
                                    carbon_intensity=ci_val)    

    c_des, c_mfg, c_ope, c_tot = eco_chip(args)
    emb = float(c_des) + float(c_mfg)
    ope = c_ope
    tot = emb + float(c_ope)


    processors.at[idx, 'Emb Crabon Most Probable'] = emb
    processors.at[idx, 'Ope Carbon Most Probable'] = ope
    processors.at[idx, 'Tot Carbon Most Probable'] = tot
    processors.at[idx, 'ECFPA Most Probable'] = emb / area

print("Most Probable CFP value calculation Finished")

Most Probable CFP value calculation Finished


# Combine Chiplet and processed processors to CSV

In [10]:
processors
chiplet['Emb Crabon Min'] = 'Insuficient Data'
chiplet['Ope Carbon Min'] = 'Insuficient Data'
chiplet['Tot Carbon Min'] = 'Insuficient Data'
chiplet['ECFPA Min'] = 'Insuficient Data'

chiplet['Emb Crabon Max'] = 'Insuficient Data'
chiplet['Ope Carbon Max'] = 'Insuficient Data'  
chiplet['Tot Carbon Max'] = 'Insuficient Data'
chiplet['ECFPA Max'] = 'Insuficient Data'

chiplet['Emb Crabon Most Probable'] = 'Insuficient Data'
chiplet['Ope Carbon Most Probable'] = 'Insuficient Data'
chiplet['Tot Carbon Most Probable'] = 'Insuficient Data'
chiplet['ECFPA Most Probable'] = 'Insuficient Data'

combined_df = pd.concat([chiplet, processors], ignore_index=True)
combined_df.to_csv('../dataset/CarbonSet.csv', index=False)
